# MovieLens Data Analysis
Bu notebook, MovieLens veri setiyle temel analizler ve görselleştirmeler içerir.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Dosya yolları
movies = pd.read_csv('../data/movies.csv')
links = pd.read_csv('../data/links.csv')
# ratings ve tags dosyaları büyük olduğu için örneklem alınabilir
ratings = pd.read_csv('../data/ratings.csv', nrows=100000)
tags = pd.read_csv('../data/tags.csv', nrows=100000)

movies.head()

## En Popüler Türler

In [ ]:
# Türleri ayır
from collections import Counter
genre_list = movies['genres'].str.split('|').sum()
genre_counts = Counter(genre_list)

plt.figure(figsize=(10,5))
sns.barplot(x=list(genre_counts.keys()), y=list(genre_counts.values()))
plt.title('En Popüler Film Türleri')
plt.ylabel('Film Sayısı')
plt.xlabel('Tür')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## Puan Dağılımı

In [ ]:
plt.figure(figsize=(8,4))
sns.histplot(ratings['rating'], bins=20, kde=True)
plt.title('Puan Dağılımı')
plt.xlabel('Puan')
plt.ylabel('Frekans')
plt.show()

## En Çok Oylanan Filmler

In [ ]:
top_movies = ratings.groupby('movieId').size().sort_values(ascending=False).head(10)
top_movies = top_movies.reset_index().merge(movies[['movieId','title']], on='movieId')
plt.figure(figsize=(10,5))
sns.barplot(x='title', y=0, data=top_movies)
plt.title('En Çok Oylanan 10 Film')
plt.ylabel('Oy Sayısı')
plt.xlabel('Film')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()